In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import metrics

# Base

In [2]:
# Importando a base
train = pd.read_csv('train_treated.csv', index_col=0)

## Dividindo entre treino e teste

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train.drop(columns=['Survived']), train['Survived'], test_size=0.25, random_state=0)

# Importando os modelos 

In [4]:
import pickle
filename = 'amostragem_scores'
infile = open(filename,'rb')
metrics_dict = pickle.load(infile)
infile.close()

In [5]:
metrics_dict.keys()

dict_keys(['Logistic GS', 'SVM GS Bagging', 'Decision GS', 'Neural GS'])

In [6]:
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."


In [7]:
def update_dict(name, predict, classifier, dictionary):
    dictionary[name] = {
        'label': name, 
        'accuracy': metrics.accuracy_score(predict, y_test), 
        'recall': metrics.recall_score(predict, y_test), 
        'precision': metrics.precision_score(predict, y_test), 
        'F1': metrics.f1_score(predict, y_test),
        'predict': predict,
        'classifier': classifier
    }
    return dictionary
    
def update_score_dict(name, predict, classifier):
    update_dict(name, predict, classifier, metrics_dict)

# Teste com votação Maioria Simples/Uniforme 

In [8]:
ensemble_predict = []
for i in range(len(x_test)):
    votes = [0, 0]
    for metric_key in metrics_dict.keys():
        predict = metrics_dict[metric_key]['predict']
        index = predict[i] if isinstance(predict[i], int) else int(predict[i].round())
        votes[index] += 1
    ensemble_predict.append(votes.index(max(votes)))

In [9]:
metrics.accuracy_score(ensemble_predict, y_test)

0.8161434977578476

In [10]:
metrics.f1_score(ensemble_predict, y_test)

0.7453416149068324

In [11]:
name = 'Votação Simples (criada)'
metrics_dict[name] = {
        'label': name, 
        'accuracy': metrics.accuracy_score(ensemble_predict, y_test), 
        'recall': metrics.recall_score(ensemble_predict, y_test), 
        'precision': metrics.precision_score(ensemble_predict, y_test), 
        'F1': metrics.f1_score(ensemble_predict, y_test),
        'predict': ensemble_predict,
        'classifier': None
    }

# Classificador de Voting 

In [12]:
from sklearn.ensemble import VotingClassifier

In [13]:
estimators=[
    ('Logistic', metrics_dict['Logistic GS']['classifier']),
    ('SVM', metrics_dict['SVM GS Bagging']['classifier']), 
    ('Decision', metrics_dict['Decision GS']['classifier']), 
    ('Neural', metrics_dict['Neural GS']['classifier'])
]

## Hard

In [14]:
voting_hard = VotingClassifier(estimators=estimators, voting='hard')

In [15]:
voting_hard.fit(x_train, y_train)
voting_predict_hard = voting_hard.predict(x_test)

In [16]:
metrics.accuracy_score(voting_predict_hard, y_test)

0.7892376681614349

In [17]:
metrics.f1_score(voting_predict_hard, y_test)

0.7006369426751592

O resultado o Voting classifier com parametro 'hard' é igual ao criado por mim no item acima

In [18]:
update_score_dict('Votação Simples (hard)', voting_predict_hard, voting_hard)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."


## Soft

In [19]:
voting_soft = VotingClassifier(estimators=estimators, voting='soft')

In [20]:
voting_soft.fit(x_train, y_train)
voting_predict_soft = voting_soft.predict(x_test)

In [21]:
update_score_dict('Votação Simples (soft)', voting_predict_soft, voting_soft)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."


O resultado do classifier com 'soft' foi um pouco melhor

In [22]:
# Ordered by F1
f1_list = sorted(list(metrics_dict.values()), key=lambda k: k['F1']) 
f1_ordered = pd.DataFrame(f1_list)
f1_ordered

,label,accuracy,recall,precision,F1,predict,classifier
0,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
3,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
6,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."


In [23]:
# Ordered by Accuracy
accuracy_list = sorted(list(metrics_dict.values()), key=lambda k: k['accuracy']) 
accuracy_ordered = pd.DataFrame(accuracy_list)
accuracy_ordered

,label,accuracy,recall,precision,F1,predict,classifier
0,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
1,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
2,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
3,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
6,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."


Mas ainda não melhor dentre todos os classificadores

# Votação com pesos (naive)

In [24]:
# colocando pesos maiores nos melhores classificadores baseados no F1
voting_weighted_naive = VotingClassifier(estimators=estimators, weights=[1.5, 0.5, 1, 1], voting='hard')

In [25]:
voting_weighted_naive.fit(x_train, y_train)
voting_weighted_naive_predict = voting_weighted_naive.predict(x_test)

In [26]:
update_score_dict('Votação com pesos naive (hard)', voting_weighted_naive_predict, voting_weighted_naive)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
7,Votação com pesos naive (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."


# Votação com pesos (normalized)

In [27]:
data = [
    metrics_dict['Logistic GS']['F1'],
    metrics_dict['SVM GS Bagging']['F1'], 
    metrics_dict['Decision GS']['F1'], 
    metrics_dict['Neural GS']['F1']
]
data = np.expand_dims(data, 1)

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)    
ret = scaler.transform(data)
ret = np.squeeze(ret)
ret += 1
ret

array([1.07373691, 1.        , 2.        , 2.        ])

In [29]:
# colocando pesos maiores nos melhores classificadores baseados no F1 (com normalizacao)
voting_weighted_normalized = VotingClassifier(estimators=estimators, weights=ret, voting='hard')

In [30]:
voting_weighted_normalized.fit(x_train, y_train)
voting_weighted_normalized_predict = voting_weighted_normalized.predict(x_test)

In [31]:
update_score_dict('Votação com pesos normalizado (hard)', voting_weighted_normalized_predict, voting_weighted_normalized)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
7,Votação com pesos naive (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
8,Votação com pesos normalizado (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."


# Score

In [32]:
# Ordered by F1
f1_list = sorted(list(metrics_dict.values()), key=lambda k: k['F1']) 
f1_ordered = pd.DataFrame(f1_list)
f1_ordered

,label,accuracy,recall,precision,F1,predict,classifier
0,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
3,Votação com pesos naive (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
4,Votação com pesos normalizado (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
5,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
7,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
8,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."


# Testando sem SVM

In [33]:
estimators=[
    ('Logistic', metrics_dict['Logistic GS']['classifier']),
    ('Decision', metrics_dict['Decision GS']['classifier']), 
    ('Neural', metrics_dict['Neural GS']['classifier'])
]

## Soft

In [34]:
voting_soft = VotingClassifier(estimators=estimators, voting='soft')

In [35]:
voting_soft.fit(x_train, y_train)
voting_predict_soft = voting_soft.predict(x_test)

In [36]:
update_score_dict('Votação simples (soft) sem SVM', voting_predict_soft, voting_soft)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
7,Votação com pesos naive (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
8,Votação com pesos normalizado (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
9,Votação simples (soft) sem SVM,0.811659,0.783784,0.690476,0.734177,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."


## Pesos normalizados

In [37]:
data_2 = [
    metrics_dict['Logistic GS']['F1'],
    metrics_dict['Decision GS']['F1'], 
    metrics_dict['Neural GS']['F1']
]
data_2 = np.expand_dims(data_2, 1)

In [38]:
scaler = MinMaxScaler()
scaler.fit(data_2)    
ret_2 = scaler.transform(data_2)
ret_2 = np.squeeze(ret_2)
ret_2 += 1
ret_2

array([1., 2., 2.])

In [39]:
# colocando pesos maiores nos melhores classificadores baseados no F1 (com normalizacao)
voting_weighted_normalized = VotingClassifier(estimators=estimators, weights=ret_2, voting='hard')

In [40]:
voting_weighted_normalized.fit(x_train, y_train)
voting_weighted_normalized_predict = voting_weighted_normalized.predict(x_test)

In [41]:
update_score_dict('Votação com pesos normalizado (hard) sem SVM', voting_weighted_normalized_predict, voting_weighted_normalized)
pd.DataFrame(metrics_dict.values())

,label,accuracy,recall,precision,F1,predict,classifier
0,Logistic GS,0.802691,0.777778,0.666667,0.717949,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","LogisticRegression(C=0.045227288910538066, cla..."
1,SVM GS Bagging,0.789238,0.728395,0.702381,0.715152,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","(SVC(C=492.1132712266245, break_ties=False, ca..."
2,Decision GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","DecisionTreeClassifier(ccp_alpha=0.0, class_we..."
3,Neural GS,0.820628,0.782051,0.726190,0.753086,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","MLPClassifier(activation='relu', alpha=1e-06, ..."
4,Votação Simples (criada),0.816143,0.779221,0.714286,0.745342,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...",None
5,Votação Simples (hard),0.789238,0.753425,0.654762,0.700637,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
6,Votação Simples (soft),0.802691,0.743902,0.726190,0.734940,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
7,Votação com pesos naive (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
8,Votação com pesos normalizado (hard),0.798206,0.753247,0.690476,0.720497,"[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
9,Votação simples (soft) sem SVM,0.811659,0.783784,0.690476,0.734177,"[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, ...","VotingClassifier(estimators=[('Logistic',\n ..."
